Import the necessary libraries...

In [3]:
from ortools.linear_solver import pywraplp

In the **second example**, we address the following optimisation problem:

$$
\begin{align*}
&\text{maximise} \quad& 7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5 \\
&\text{subject to} \quad& A\boldsymbol{x}\leq b \\
&& x_i \in \mathbb{N}_0, \ i=1,\dots,5
\end{align*}
$$
where
$$A = 
\begin{bmatrix}
    5 & 7 & 9 & 2 & 1 \\
    18 & 4 & -9 & 10 & 12 \\
    4 & 7 & 3 & 8 & 5 \\
    5 & 13 & 16 & 3 & -7
\end{bmatrix}$$
and
$$b=[250, 285, 211, 315]^T$$

Here is the data:

In [4]:
data = {}
data['constraint_coeffs'] = [
    [5, 7, 9, 2, 1],
    [18, 4, -9, 10, 12],
    [4, 7, 3, 8, 5],
    [5, 13, 16, 3, -7],
]
data['bounds'] = [250, 285, 211, 315]
data['obj_coeffs'] = [7, 8, 2, 9, 6]
data['num_vars'] = 5
data['num_constraints'] = 4


We thus proceed through the steps to *create*, *specify* and *solve* a **mixed integer programming** problem using Google's OR-Tools.

### 1. Create Solver
First, create a solver for MIP programming, as a class instance:

In [5]:
solver = pywraplp.Solver.CreateSolver(solver_id='SCIP')

SCIP is the back-end for Google's solver. Remember that you can leverage solvers from external vendors.

### 2. Create Variables
Next, create the integer-valued variables $$x_i \in \mathbb{N}_0, \ i=1,\dots,5$$ whose values we wish to optimise, in a loop:

In [6]:
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, f"x[{str(j)}]")

### 3. Create Constrains
Specify the linear constrains in $$A\boldsymbol{x}\leq b.$$ We do this in a loop:

In [7]:
for i in range(data['num_constraints']):
    this_constrains = data['constraint_coeffs'][i]
    constraint_expr = [
        this_constrains[j] * x[j] for j in range(data['num_vars'])
    ]
    solver.Add(sum(constraint_expr) <= data['bounds'][i])

### 4. Objective Function
Create the objective function $$7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5,$$ using list comprehension:

In [8]:
obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]

### 5. Execute Job
Solve the problem by setting the maximisation task and running the solver. A ```status``` informs us whether the optimisation routine reached an Optimal (```=0```) or Feasible (```=1```) solution.

In [ ]:
solver.Maximize(sum(obj_expr))
status = solver.Solve()
print(f"The optimisation routine returned status {status}")

Here is a summary of the model you have optimised:

Extract the optimal objective and variable values:

In [ ]:
print('Objective value =', solver.Objective().Value())
for j in range(data['num_vars']):
    print(x[j].name(), ' = ', x[j].solution_value())
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())